In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import scipy
import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV


In [ ]:
train_df = pd.read_csv('/kaggle/input/mercedes-benz-greener-manufacturing/train.csv.zip')
test_df = pd.read_csv('/kaggle/input/mercedes-benz-greener-manufacturing/test.csv.zip')


In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
train_df.shape, test_df.shape

In [ ]:
train_df['y'].describe()

In [ ]:
train_df['y'].hist();

In [ ]:
sns.boxplot(train_df['y']);

In [ ]:
#removing of outliars 


train_df = train_df.drop(train_df[train_df['y'] > 140].index)

In [ ]:
sns.boxplot(train_df['y']);

In [ ]:
train_df.dtypes.value_counts()

In [ ]:
cat_feature = train_df.select_dtypes(include='object')
cat_feature

In [ ]:
fig = plt.figure(figsize=(15, 10))

for index, col in enumerate(cat_feature):
    plt.subplot(4, 2, index+1)
    sns.countplot(cat_feature[col])
   

    
    
plt.tight_layout()    
plt.show(); 

In [ ]:
fig = plt.figure(figsize=(20, 35))

for index, col in enumerate(cat_feature):
    plt.subplot(4, 2, index+1)
    sns.boxplot(data=train_df, x=col, y='y')
   

    
    
plt.tight_layout()    
plt.show(); 

In [ ]:
data = pd.concat([train_df, test_df])


In [ ]:
data[['X0', 'X1', 'X2']]

In [ ]:
def code_сount(dataset, feature):
    dict = dataset[feature].value_counts().to_dict()
    dataset[feature] = dataset[feature].map(dict)
    
    return dataset[feature] 
        

In [ ]:
for feature in data[['X0', 'X1', 'X2']]:
    code_сount(data, feature)

In [ ]:
'''data = pd.concat([data, pd.get_dummies(data[['X0', 'X1', 'X2']])], axis=1)'''

In [ ]:
data = data.drop(columns=['X3', 'X4', 'X5', 'X6', 'X8'])

In [ ]:
'''data.drop(columns=['X0', 'X1', 'X2'], inplace=True)'''

In [ ]:
data.info()

In [ ]:
num_features = train_df.select_dtypes(exclude='object').head()
num_features.head()

In [ ]:
'''num_features.describe()'''

In [ ]:
'''num_nul_var = []
for col in num_features:
    if train_df[col].var() == 0:
        num_nul_var.append(col)
        print(col, train_df[col].var())'''

In [ ]:
'''data.drop(columns=num_nul_var, inplace=True)'''

In [ ]:
train_df = data[:train_df.shape[0]]

In [ ]:
test_df =  data[train_df.shape[0]:]


In [ ]:
X = train_df.drop(columns=['ID', 'y'])
y = train_df['y']
X_test = test_df.drop(columns=['ID', 'y'])


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42)

In [ ]:
rf = RandomForestRegressor(max_depth=5, min_samples_leaf=3, n_estimators=145,
                      n_jobs=-1, random_state=1)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_val)

r2_score(y_val, y_pred)

In [ ]:
'''params= {'max_depth': list(range(5,21, 1)), 
              'min_samples_leaf': list(range(1,6, 1)),
              'n_estimators': [145], 'n_jobs': [-1], 'random_state': [1]}


cv_rf = GridSearchCV(RandomForestRegressor(), params)

cv_rf.fit(X_train, y_train)
y_pred = cv_rf.predict(X_val)

r2_score(y_val, y_pred), cv_rf.best_estimator_'''

In [ ]:
model = RandomForestRegressor(max_depth=5, min_samples_leaf=3, n_estimators=145,
                      n_jobs=-1, random_state=1)

model.fit(X, y)
predict = model.predict(X_test)

In [ ]:
output = pd.DataFrame({'ID': test_df.ID, 'y': predict})
output.to_csv('mercedes_benz_baseline.csv', index=False)